In [8]:
%reset -f

In [9]:
import os
import pandas as pd
rootPath = "D:\\kelp"
train_folder = os.path.join(rootPath, "train_satellite")
train_label = os.path.join(rootPath, "train_kelp")
metadata = pd.read_csv(os.path.join(rootPath, "metadata_fTq0l2T.csv"))

In [17]:
import rasterio
import numpy as np
import tensorflow as tf
from keras.layers import Layer, Conv2D, Concatenate, MaxPooling2D, Dropout, Conv2DTranspose, Activation, Reshape, ConvLSTM2D, Input, BatchNormalization
from keras.models import Model
from keras.utils import plot_model
import cv2
import matplotlib.pyplot as plt
from PIL import Image

In [11]:
train_image_list = []
train_mask_list = []
predict_image_list = []

for index, row in metadata.iterrows():
    filename = row['filename']
    if row["in_train"] == True and row["type"] == "satellite":
        train_image_list.append(filename)
    elif row["in_train"] == False and row["type"] == "kelp":
        train_mask_list.append(filename)
    else:
        predict_image_list.append(filename)

In [19]:
train_image_list.sort()
train_mask_list.sort()
predict_image_list.sort()

In [12]:
with rasterio.open(os.path.join(train_folder,train_image_list[1])) as src:
    data = src.read()  # Reads the raster data as a numpy array


c:\Users\guoyy\miniconda3\envs\tfgpu\lib\site-packages\rasterio\__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [13]:
def doubleconv2d(input_size, filter):
    temp = Conv2D(filter, 3, padding= "same", activation="relu", kernel_initializer= "he_normal")(input_size)
    output = Conv2D(filter, 3, padding= "same", activation="relu", kernel_initializer= "he_normal")(temp)
    return output

def down2d(input_size, filter):
    mapping = doubleconv2d(input_size, filter)
    temp2 = MaxPooling2D(2)(mapping)
    output = Dropout(0.1)(temp2)
    return mapping, output

def un2d(input_size, mapping, filter, shape):
    TransConv = Conv2DTranspose(filter, 3, 2, padding="same")(input_size)
    BN = BatchNormalization(axis=3)(TransConv)
    ACT = Activation('relu')(BN)
    
    mapping = tf.image.resize_with_crop_or_pad(mapping, np.int32(shape),np.int32(shape))
    ACT = tf.image.resize_with_crop_or_pad(ACT, np.int32(shape),np.int32(shape))

    MAP = Reshape(target_shape=(1, np.int32(shape), np.int32(shape), filter))(mapping)
    UP = Reshape(target_shape=(1, np.int32(shape), np.int32(shape), filter))(ACT)
    UP = Concatenate(axis = 1)([MAP,UP])
    UP = ConvLSTM2D(filters = filter, 
                            kernel_size=(3, 3), 
                            padding='same', 
                            return_sequences = False, 
                            go_backwards = True, 
                            kernel_initializer = 'he_normal')(UP)
    UP = Dropout(0.1)(UP)
    UP = doubleconv2d(UP, filter)
    return UP

def make_unet(input_size = (350, 350, 4)):
    inputs = Input(input_size)
    map1, down1 = down2d(inputs, 64)
    map2, down2 = down2d(down1, 128)
    map3, down3 = down2d(down2, 256)
    
    bottleneck1 = doubleconv2d(down3, 512)
    bottleneck2 = doubleconv2d(bottleneck1, 512)
    merge_dense = Concatenate(axis=3)([bottleneck1,bottleneck2])
    bottleneck3 = doubleconv2d(merge_dense, 512)

    up1 = un2d(bottleneck3, map3, 256, input_size[0]/4)
    up2 = un2d(up1, map2, 128, input_size[0]/2)
    up3 = un2d(up2, map1, 64, input_size[0])

    outputs = Conv2D(1, 1, padding="same", activation = "sigmoid")(up3)
    unet_model = Model(inputs, outputs, name="U-Net")
    
    return unet_model

In [14]:
u_net = make_unet()

In [15]:
u_net.summary()

Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 350, 350, 4  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 350, 350, 64  2368        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 350, 350, 64  36928       ['conv2d[0][0]']                 
                                )                                                             

In [18]:
%%capture

# Generate the plot
plot_model(u_net, to_file='u_net_model.png', show_shapes=True, show_layer_names=True)

In [26]:
with rasterio.open(os.path.join(train_folder,train_image_list[1])) as src:
    data = src.read()  # Reads the raster data as a numpy array

type(data[[0,1,5,6]])

c:\Users\guoyy\miniconda3\envs\tfgpu\lib\site-packages\rasterio\__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


numpy.ndarray

In [31]:
len(train_image_list)

5635

In [32]:
len(train_mask_list)

5635

In [40]:
def make_ince_dataset(validation = False):
    x, y = [], []
    if validation:
        for i, (image, mask) in enumerate(zip(train_image_list[1000:1201], train_mask_list[1000:1201])):
            print("\r"+str(i+1)+"/"+str(1200),end="")
            with rasterio.open(os.path.join(train_folder, train_image_list[i])) as ima:
                image = ima.read()
            image = image[[0,1,5,6]]
            
            with rasterio.open(os.path.join(train_label, train_mask_list[i])) as ma:
                mask = ma.read()
                
            image = (image-np.min(image))/(np.max(image) - np.min(image) + 1e-10)
            mask = (mask-np.min(mask))/(np.max(mask) - np.min(mask) + 1e-10)

            x.append(image)
            y.append(mask)
    else:
        for i, (image, mask) in enumerate(zip(train_image_list[:1000], train_mask_list[:1000])):
            print("\r"+str(i+1)+"/"+str(1000),end="")
            with rasterio.open(os.path.join(train_folder, train_image_list[i])) as ima:
                image = ima.read()
            image = image[[0,1,5,6]]
            
            with rasterio.open(os.path.join(train_label, train_mask_list[i])) as ma:
                mask = ma.read()
                
            image = (image-np.min(image))/(np.max(image) - np.min(image) + 1e-10)
            mask = (mask-np.min(mask))/(np.max(mask) - np.min(mask) + 1e-10)

            x.append(image)
            y.append(mask)
    return np.array(x), np.array(y)
    
    

In [36]:
x,y = make_ince_dataset()
v_x,v_y = make_ince_dataset(True)

2/1000

c:\Users\guoyy\miniconda3\envs\tfgpu\lib\site-packages\rasterio\__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


201/12000

In [39]:
x[1].shape

(4, 350, 350)

### trainning the model

In [ ]:
u_net.compile(
    optimizer=
)

**transfer our image data into two types of tensor and the label into another tensor**
1. (350, 350, 4) for infrared, near infrared, cloud mask, elevation. we call that "ince"
2. (350, 350, 3) for RGB, we call that "rgb"
3. (350, 350, 1) for label

**create DataLoader for each type of dataset**

**I need to consider the "ince" and the "rgb" data**

| Dataloader Type    | Dataloader Name |
|-------------------|-----------------|
| Train Dataloader  | 1. train_ince_dl |
|                   | 2. train_rgb_dl  |
| Val Dataloader    | 3. val_ince_dl   |
|                   | 4. val_rgb_dl    |
| Test Dataloader   | 5. test_ince_dl  |
|                   | 6. test_rgb_dl   |


In [14]:
train_ince_dl_temp       =     TrainDataset(train_dir = train_dir, label_dir = label_dir, type = "train", val_size = 0.2, sep = "ince")
train_rgb_dl_temp        =     TrainDataset(train_dir = train_dir, label_dir = label_dir, type = "train", val_size = 0.2, sep = "rgb")
val_ince_dl_temp         =     TrainDataset(train_dir = train_dir, label_dir = label_dir, type = "val", val_size = 0.2, sep = "ince")
val_rgb_dl_temp          =     TrainDataset(train_dir = train_dir, label_dir = label_dir, type = "val", val_size = 0.2, sep = "rgb")
test_ince_rgb_dl_temp    =     TestDataset(test_dir = test_dir)

In [15]:
train_ince_dl       =     DataLoader(train_ince_dl_temp, batch_size = 8)
train_rgb_dl        =     DataLoader(train_rgb_dl_temp, batch_size = 8)
val_ince_dl         =     DataLoader(val_ince_dl_temp, batch_size = 8)
val_rgb_dl          =     DataLoader(val_rgb_dl_temp, batch_size = 8)
test_ince_rgb_dl    =     DataLoader(test_ince_rgb_dl_temp, batch_size = 8)

**create Unet and training function**

*allow me to make an estimation first and inspection usnig torchsummary first*

- initiate the model

In [16]:
incemodel = UShapeNet(4).to(device=device)
# summary(incemodel, (4,350,350));
rgbmodel = UShapeNet(3).to(device=device)
# summary(rgbmodel, (3,350,350));

**define the training process!  together with our DataLoader**
- training ince model 

In [17]:
loss_fn = CombinedLoss(weight_dice=0.5, weight_ce=0.5)
optimizer = torch.optim.Adam(incemodel.parameters(), lr=0.001)
num_epochs = 5
train_accuracies = []
train_losses = []
val_accuracies = []
for epoch in trange(num_epochs, desc="out of epochs", leave=True):
    train_epoch_accuracy_list = []
    train_epoch_batch_loss = []
    val_epoch_accuracy_list = []
    for index, batch in enumerate(tqdm(iter(train_ince_dl))):
        x, y = batch
        batch_loss = train_batch(x, y, incemodel, loss_fn, optimizer)
        train_epoch_batch_loss.append(batch_loss)
        
    for index, batch in enumerate(tqdm(iter(train_ince_dl))):
        x, y = batch
        temp = accuracy(x, y, incemodel)
        train_epoch_accuracy_list.append(temp)
    
    for index, batch in enumerate(tqdm(iter(val_ince_dl))):
        x, y = batch
        temp = accuracy(x, y, incemodel)
        val_epoch_accuracy_list.append(temp)
        
    train_accuracies.append(np.mean(train_epoch_accuracy_list))
    train_losses.append(np.mean(train_epoch_batch_loss))
    val_accuracies.append(np.mean(val_epoch_accuracy_list))
    
print(f"train_accuracies is {train_accuracies}")
print(f"train_losses is {train_losses}")
print(f"val_accuracies is {val_accuracies}")

out of epochs:  20%|██        | 1/5 [00:11<00:45, 11.30s/it]


KeyboardInterrupt: 

- save the ince model

In [ ]:
torch.save(incemodel.state_dict(), os.path.join(modelPath,'ince_model_params.pth'))

- train rgb model

In [ ]:
loss_fn = CombinedLoss(weight_dice=0.5, weight_ce=0.5)
optimizer = torch.optim.Adam(rgbmodel.parameters(), lr=0.001)
num_epochs = 5
train_accuracies = []
train_losses = []
val_accuracies = []
for epoch in trange(num_epochs, desc="out of epochs", leave=True):
    train_epoch_accuracy_list = []
    train_epoch_batch_loss = []
    val_epoch_accuracy_list = []
    for index, batch in enumerate(tqdm(iter(train_rgb_dl))):
        x, y = batch
        batch_loss = train_batch(x, y, rgbmodel, loss_fn, optimizer)
        train_epoch_batch_loss.append(batch_loss)
        
    for index, batch in enumerate(tqdm(iter(train_rgb_dl))):
        x, y = batch
        temp = accuracy(x, y, rgbmodel)
        train_epoch_accuracy_list.append(temp)
    
    for index, batch in enumerate(tqdm(iter(val_rgb_dl))):
        x, y = batch
        temp = accuracy(x, y, rgbmodel)
        val_epoch_accuracy_list.append(temp)
        
    train_accuracies.append(np.mean(train_epoch_accuracy_list))
    train_losses.append(np.mean(train_epoch_batch_loss))
    val_accuracies.append(np.mean(val_epoch_accuracy_list))
    
print(f"train_accuracies is {train_accuracies}")
print(f"train_losses is {train_losses}")
print(f"val_accuracies is {val_accuracies}")

out of epochs: 100%|██████████| 5/5 [00:27<00:00,  5.57s/it]

train_accuracies is [0.59146464, 0.992591, 0.992591, 0.992591, 0.992591]
train_losses is [0.525196635723114, 1.7771829605102538, 0.28945243954658506, 0.28411559462547303, 0.2776166796684265]
val_accuracies is [0.54165256, 0.99517804, 0.99517804, 0.99517804, 0.99517804]


- save the rgb model

In [ ]:
torch.save(rgbmodel.state_dict(), os.path.join(modelPath,'rgb_model_params.pth'))

**vaccuracy and loss**
- it is super good after one epic

In [ ]:
train_accuracies

[0.59146464, 0.992591, 0.992591, 0.992591, 0.992591]

In [ ]:
val_accuracies

[0.54165256, 0.99517804, 0.99517804, 0.99517804, 0.99517804]

- model combined the rgb and ince together!

    how to combine the (350, 350, 2) and (350, 350, 2) together??

    It is in the [test.ipynb](https://github.com/y1u2a3n4g5/Multi-Layer-Unet/blob/main/test.ipynb)

visulization of the Unet model

In [ ]:
# from torchviz import make_dot
# rgbmodel.eval()
# for i in rgbmodel.parameters():
#     i.requires_grad = True
# dummy_input = torch.randn(8, 3, 350, 350).to(device=device)
# y = rgbmodel(dummy_input)
# graph = make_dot(y, params=dict(rgbmodel.named_parameters()))
# graph.render('rgbmodel', format='png')

In [ ]:
from torchviz import make_dot
incemodel.eval()
for i in incemodel.parameters():
    i.requires_grad = True
dummy_input = torch.randn(8, 4, 350, 350).to(device=device)
y = incemodel(dummy_input)
graph = make_dot(y, params=dict(incemodel.named_parameters()))
graph.render('incemodel', format='png')

'incemodel.png'